In [2]:
import string
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

# Load data
df = pd.read_csv("stories.csv", encoding='utf-8')
story_list = df.stories.to_list()
texts = ''.join(story_list[0:25])  # Using 25 stories for testing
text = [[char.lower() for char in text if char in string.printable] for text in texts]
# text = [[char for char in text if char not in string.punctuation] for text in texts]
text = ''.join([j for i in text for j in i])
print(f'Corpus Length: {len(text)}')
# print(text)

Corpus Length: 754861


In [3]:
# Creating character / word mappings

# Word mappings
words = sorted(list(set(text.split())))
print(f'Unique words: {len(words)}')

n_to_word = {n:word for n, word in enumerate(words)}
word_to_n = {word:n for n, word in enumerate(words)}

Unique words: 15511


In [4]:
# Splitting the dataset to test/train
X = []
Y = []

length = len(text.split())

# Length of extracted word mapping sequences
seq_length = 100

# Sets are 100 words, followed by the target word
for i in range(0, length-seq_length, 1):
    sequence = text.split()[i:i + seq_length]
    label = text.split()[i + seq_length]
    X.append([word_to_n[word] for word in sequence])
    Y.append(word_to_n[label])
n_patterns = len(X)
print(f'Total patterns: {n_patterns})

In [5]:
# Transform to appropriate shape LSTM (samples, time steps, features)
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified/float(len(words))
Y_modified = np_utils.to_categorical(Y)

In [8]:
# Define LSTM Model
model = Sequential()
model.add(LSTM(100, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 100)          40800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15511)            

In [9]:
# Fit Model
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)

filename = '100_0.2_100_0.2_100_0.2_e100_b50.h5'
model.save_weights(filename)

Epoch 1/100
2841/2841 [==============================] - 467s 160ms/step - loss: 7.3393 - accuracy: 0.0483
Epoch 2/100
2841/2841 [==============================] - 457s 161ms/step - loss: 7.0080 - accuracy: 0.0495
Epoch 3/100
2841/2841 [==============================] - 463s 163ms/step - loss: 6.9859 - accuracy: 0.0499
Epoch 4/100
2841/2841 [==============================] - 457s 161ms/step - loss: 6.9812 - accuracy: 0.0507
Epoch 5/100
2841/2841 [==============================] - 461s 162ms/step - loss: 6.9786 - accuracy: 0.0499
Epoch 6/100
2841/2841 [==============================] - 457s 161ms/step - loss: 6.9790 - accuracy: 0.0484
Epoch 7/100
2841/2841 [==============================] - 461s 162ms/step - loss: 6.9588 - accuracy: 0.0493
Epoch 8/100
2841/2841 [==============================] - 457s 161ms/step - loss: 6.9124 - accuracy: 0.0500
Epoch 9/100
2841/2841 [==============================] - 457s 161ms/step - loss: 6.8809 - accuracy: 0.0483
Epoch 10/100
2841/2841 [=============

In [10]:
# Generate texts with last saved fit
model.load_weights(filename)

string_mapped = X[99]
full_string = [n_to_word[value] for value in string_mapped]

# Generation
for i in range(100):
    x = np.reshape(string_mapped, (1, len(string_mapped), 1))
    x = x / float(len(words))
    
    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [n_to_word[value] for value in string_mapped]
    full_string.append(n_to_word[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
# Combine generated text and output
txt = ""
for word in full_string:
    txt = txt+' '+word
print(txt)

 boilover. maybe the attacker truly believed his life was in danger and mistook the slightest movement as the beginning of a punch. perhaps he just let his anger at the situation get the best of him. maybe a mix of both. whats the objective truth here? and for whom is that truth valid? when youre a cop, understanding these nuances in truth is critical. and, understanding the power those nuances can have is even more important. not only when we have to discern whether or not someones recollection of events is accurate, but when we lie to achieve a specific loss. the the i the i the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the i the
